In [1]:
import time
import numpy as np
from xarm.wrapper import XArmAPI
from motion import fish_motion

SDK_VERSION: 1.13.30


In [2]:

arm = XArmAPI('192.168.1.217')
arm.connect()


ROBOT_IP: 192.168.1.217, VERSION: v2.3.0, PROTOCOL: V1, DETAIL: 7,7,XS1304,AC1303,v2.3.0, TYPE1300: [1, 1]
change protocol identifier to 3


In [3]:

amplitude=5
frequency=1
duration = 5

yaw_amplitude=5
yaw_frequency=frequency
phase_shift = 0

steps = fish_motion(arm, amplitude, frequency, yaw_amplitude, yaw_frequency, phase_shift, duration)


[set_state], xArm is ready to move


In [12]:
arm.set_state(0)
arm.get_state()


[set_state], xArm is ready to move


(0, 2)

In [ ]:

arm.disconnect()